In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import pickle

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
with open("all_data.pickle", "rb") as f:
    d = pickle.load(f)

In [ ]:
list(d.keys())

In [ ]:
measurements, regionprops, labels, img = d[
    "/n/scratch2/jqs1/fidelity/190301/jqs_photobleach_100ms_de32_mkate2_0004.nd2"
]

In [ ]:
# regionprops.reset_index(inplace=True)
regionprops.head()

In [ ]:
%%output size=150
#%%opts Image {+axiswise}
hv.Image(img / img.max()).options(cmap="gray") + hv.Image(labels != 0).options(
    cmap="blues"
)

# Plotting

In [ ]:
list(measurements.keys())

In [ ]:
traces = []
rp_list = []
rp_df = pd.DataFrame()
for i in range(7):
    measurements, regionprops, labels, img = d[
        "/n/scratch2/jqs1/fidelity/190301/jqs_photobleach_100ms_de32_mkate2_000"
        + str(i)
        + ".nd2"
    ]
    traces.append(measurements["mean"][1:])
    rp_list.append(regionprops)
traces = np.concatenate(traces)
rp_df = pd.concat(rp_list, sort=False)
rp_df.reset_index(inplace=True)
print(traces.shape)

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps

# G = 10


df = pd.DataFrame(traces)  # + np.random.normal(0,1,Gsamp.T.shape))
bins = np.arange(df[0].min() - 1, df[0].max() + 1, 70)
df["bin"] = pd.cut(df[0], bins=bins)
rp_df["bin"] = df["bin"]
rp_df = rp_df[df[0] > 1000]
df = df[df[0] > 1000]
x_min, x_max, y_min, y_max = [500, 1500, 500, 1700]
df = df[
    (x_max > rp_df.centroid_x)
    & (rp_df.centroid_x > x_min)
    & (y_max > rp_df.centroid_y)
    & (rp_df.centroid_y > y_min)
]
# rp_df = rp_df[(1500 > rp_df.centroid_x) & (rp_df.centroid_x > 500) & (1500 > rp_df.centroid_y) & (rp_df.centroid_y > 500)]

p = df.apply(lambda x: x[:-1:] / x[0], axis=1)
p["bin"] = df["bin"]

tau = 70
pbar = p.groupby(p["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
mu = df.groupby(df["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
sigma2 = df.groupby(df["bin"]).var(ddof=0)[df.groupby(df["bin"]).size() > tau]

y = sigma2.div(mu[0].values, axis="rows")
x = pbar
r = y.values.flatten() / x.values.flatten()

# linfit = np.polyfit(x.values.flatten(),y.values.flatten(),1)
# linfit_fn = np.poly1d(linfit)

plt.figure(figsize=(12, 8))
# plt.scatter(x,y)

xfit = np.linspace(0, 0.25, 100)
nu_bar = np.round(np.mean(r[np.isfinite(r)]), 2)
# plt.plot(xfit,nu_bar*xfit,'-r',linewidth=6)

plt.title(
    r"$\nu = \frac{\hat{\sigma}^2}{f_{max}\hat{p}(1-\hat{p})}$ =" + str(nu_bar),
    fontsize=20,
    pad=20,
)
plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
bin_hist = df.groupby(df["bin"]).size()
print(bin_hist[bin_hist.values > tau])
print(r"mean ratio: ", np.mean(r[np.isfinite(r)]))
print("median ratio: ", np.median(r[np.isfinite(r)]))

nu_int = []
nu_dict = {}
cmap = cm.get_cmap("coolwarm")
imax = x.index[-1].left
imin = x.index[0].left
q = 1
cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
# plt.vlines(q,0,3)

for name, group in x.groupby("bin"):
    #     nu_int.append(-6*simps(y.loc[name].values,x.loc[name].values))
    dp = x.loc[name].values

    dp = dp[x.loc[name].values > 1 - q]
    f = y.loc[name].values[x.loc[name].values > 1 - q]
    nu_int.append(cq * simps(f, dp))
    c = (name.left - imin) / (imax - imin)
    nu_dict[name] = nu_int[-1]
    plt.scatter(1 - x.loc[name].values, y.loc[name].values, color=cmap(c), label=name)
#     plt.legend()
print(nu_int, np.mean(nu_int))
plt.title(
    r"$\nu = "
    + str(np.round(-cq))
    + r"\cdot \int\frac{\hat{\sigma}^2}{f_{max}}dp$ ="
    + str(np.round(np.mean(nu_int), 2)),
    fontsize=20,
    pad=20,
)

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": df[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
I0 = traces[:, 0]
plt.hist(I0, bins=30)
print(np.mean(I0), np.var(I0))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cmap = cm.get_cmap("coolwarm")
fig = plt.figure(figsize=(12, 8))
# ax = fig.add_subplot(111, projection='3d')

# rp_df = regionprops[regionprops.groupby(regionprops['bin']).size() > tau]
for name, group in rp_df.groupby("bin"):
    if group.shape[0] > tau:
        #         print(group.shape[0])
        #         print(name,nu_dict[name] )
        #         c = (name.left - imin)/(imax - imin)
        c = (nu_dict[name] - min(nu_int)) / (max(nu_int) - min(nu_int))
        z = name.left * np.ones(group.centroid_x.shape)
        #     print(group.centroid_x)
        plt.scatter(
            group.centroid_x, group.centroid_y, color=cmap(c), label=nu_dict[name]
        )
#         ax.scatter(group.centroid_x, group.centroid_y,z, c=cmap(c))


#         if nu_dict[name] > 17.5:
#             plt.scatter(group.centroid_x,group.centroid_y, lcolor='red',label=nu_dict[name])
#         if 17.5 > nu_dict[name] > 12.5:
#             plt.scatter(group.centroid_x,group.centroid_y, color='black',label=nu_dict[name])
#         if  nu_dict[name] < 12.5:
#             plt.scatter(group.centroid_x,group.centroid_y, color='blue',label=nu_dict[name])

# plt.legend()